<a href="https://colab.research.google.com/github/SHEETALDHARASHAN/Heart-Disease-prediction-ml-model/blob/main/HDP_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load the dataset
file_path = 'heart_disease_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())

# Preprocess the data
# Handle missing values (if any)
data.fillna(data.mean(), inplace=True)

# Separate features and target
# Assuming the last column is the target and named 'target'
X = data.drop('target', axis=1)
y = data['target']

# Normalize/scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize the models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'k-NN': KNeighborsClassifier(n_neighbors=5)
}

# Train and evaluate each model
best_model = None
best_score = 0
evaluation_results = {}

for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    evaluation_results[model_name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': conf_matrix
    }

    # Cross-validation
    cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    cv_score_mean = cv_scores.mean()

    print(f"\n{model_name} Evaluation Metrics:")
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Cross-Validation Accuracy: {cv_score_mean}')

    # Update the best model based on cross-validation score
    if cv_score_mean > best_score:
        best_score = cv_score_mean
        best_model = model

print("\nBest model based on cross-validation accuracy:")
print(best_model)

# Function to predict heart disease probability for new data input
def predict_heart_disease(input_data, model):
    # Convert the input data to a DataFrame
    input_df = pd.DataFrame([input_data], columns=X.columns)

    # Scale the input data
    input_scaled = scaler.transform(input_df)

    # Get the probability of heart disease
    probability = model.predict_proba(input_scaled)[0][1]

    return probability

# Example usage with the best model
new_patient_data = {
    'age': 48,
    'sex': 1,  # 1: male, 0: female
    'cp': 0,  # chest pain type
    'trestbps': 1300,  # resting blood pressure
    'chol': 256,  # serum cholesterol
    'fbs': 1,  # fasting blood sugar > 120 mg/dl
    'restecg': 0,  # resting electrocardiographic results
    'thalach': 150,  # maximum heart rate achieved
    'exang': 1,  # exercise-induced angina
    'oldpeak': 0,  # ST depression induced by exercise relative to rest
    'slope': 2,  # the slope of the peak exercise ST segment
    'ca': 2,  # number of major vessels colored by fluoroscopy
    'thal': 3  # thalassemia
}

# Predict the probability with the best model
probability = predict_heart_disease(new_patient_data, best_model)
print(f'The probability of having heart disease is {probability * 100:.2f}%')


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   48    1   0       130   256    1        0      150      1      0.0      2   
1   61    1   0       148   203    0        1      161      0      0.0      2   
2   44    0   2       118   242    0        1      149      0      0.3      1   
3   47    1   0       110   275    0        0      118      1      1.0      1   
4   56    1   3       120   193    0        0      162      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   1     3       0  
2   1     2       1  
3   1     2       0  
4   0     3       1  

Logistic Regression Evaluation Metrics:
Accuracy: 0.819672131147541
Precision: 0.8205128205128205
Recall: 0.8888888888888888
F1 Score: 0.8533333333333333
Confusion Matrix:
[[36 14]
 [ 8 64]]
Cross-Validation Accuracy: 0.8283565912477984

Random Forest Evaluation Metrics:
Accuracy: 0.9508196721311475
Precision: 0.9230769230769231
Recall: 1.0
F1 Score: 0.9600000000000001
Confusio